In [5]:
import inspect
from functools import wraps


def get_default_args(func):
    signature = inspect.signature(func)
    return {
        k: v.default
        for k, v in signature.parameters.items()
        if v.default is not inspect.Parameter.empty
    }


def block(func):
    @wraps(func)
    def filter_kwargs(*args, **kwargs):
        # FIXME: make this assert more nice
        # replace with printing info that we accept only named variables
        assert len(args) == 0
        print(func.__name__, func.__code__.co_varnames)
        default_args = get_default_args(func)
        print(kwargs)
        selected_kwargs = {}
        for key in func.__code__.co_varnames:
            # FIXME: make this assert more nice
            # e. g., display a message that graph doesn't compile
            assert (key in kwargs) or (key in default_args)
            if key in kwargs:
                selected_kwargs[key] = kwargs[key]
            else:
                selected_kwargs[key] = default_args[key]
        for key in kwargs:
            # TODO: make this configurable
            assert key in func.__code__.co_varnames
        result = func(**selected_kwargs)
        # FIXME: make this assert more nice
        # e. g., display a message that graph doesn't compile
        assert type(result) is dict
        # TODO: register function, inputs and outputs
        # TODO: how to call a fuction by name
        # https://stackoverflow.com/questions/3061/calling-a-function-of-a-module-by-using-its-name-a-string
        return result
    return filter_kwargs


@block
def f(a, b=4):
    return {'sum': a + b}

# f(a=2, c=3)
f(a=2, b=3)


f ('a', 'b')
{'a': 2, 'b': 3}
keys dict_keys(['sum'])


{'sum': 5}

In [1]:
# simulator

# TODO: time tracker
# TODO: blocks
graph = build_graph(blocks)  # TODO: use all available blocks
# TODO: or maybe we could also build custom graph with mini-notation
for t in range(10):
    run_graph(t)  # TODO

# we should probably also have other options to run the graph
# in some other regimes
# e. g., some kind of live-coding-loop regime
# or depending on [0, 1] time
# or position in bar or 4-bar cycle
# or just having infinite stream, not depending on anything in particular

NameError: name 'build_graph' is not defined

In [1]:
class Config():
    def __init__(self):
        pass

    def __getattr__(self, name):
        if name[0] == '_':
            setattr(self, name, self)
            def wrapper(*args, **kwargs):
                assert len(args) == 1
                assert not kwargs
                self[name[1:]] = args[0]
                return self
            return wrapper
        else:
            return self[name]

    def __setitem__(self, key, value):
        setattr(self, key, value)

m = Config()
m._amp(4)
print(m.amp)

4


In [6]:
# multiple dispatch example in python

from functools import singledispatchmethod

class Sum:

    @singledispatchmethod
    def sumMethod(self, arg1, arg2):
        print("Default implementation with arg1 = %s and arg2 = %s" % (arg1, arg2))

    @sumMethod.register(int)
    def _(self, arg1, arg2):
        print("Sum with arg1 as integer. %s + %s = %s" % (arg1, arg2, arg1 + arg2))

    @sumMethod.register(float)
    def _(self, arg1, arg2):
        print("Sum with arg1 as float. %s + %s = %s" % (arg1, arg2, arg1 + arg2))


s = Sum()
s.sumMethod(2, 3)
s.sumMethod(2.1, 3.4)
s.sumMethod("hi", 3.4)


Sum with arg1 as integer. 2 + 3 = 5
Sum with arg1 as float. 2.1 + 3.4 = 5.5
Default implementation with arg1 = hi and arg2 = 3.4
